In [1]:
import numpy as np
import torch
import sklearn
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from time import time
import matplotlib.pyplot as plt

In [2]:
cifar = fetch_openml('cifar_10')

In [6]:
X, y = cifar.data.reshape(len(cifar.data), 3, 32, 32), cifar.target

In [7]:
X.shape

(60000, 3, 32, 32)

In [8]:
y.shape

(60000,)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)